In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.parser import parse

In [3]:
#import airplane crashes csv into a datframe
airline_crash_csvfile = "db/Airplane_Crashes_and_Fatalities_Since_1908.csv"

airline_crashes_df = pd.read_csv(airline_crash_csvfile, error_bad_lines=False,warn_bad_lines=False)
airline_crashes_df.head()

FileNotFoundError: [Errno 2] File b'db/Airplane_Crashes_and_Fatalities_Since_1908.csv' does not exist: b'db/Airplane_Crashes_and_Fatalities_Since_1908.csv'

In [7]:
#Create new dataframe with select columns
airplane_crashes_df = airline_crashes_df[['Ac_id','Date','Time','Location','Operator','Type','Aboard','Fatalities','Ground','Summary']]
airplane_crashes_df.head()


,Ac_id,Date,Time,Location,Operator,Type,Aboard,Fatalities,Ground,Summary
0,1,9/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,Wright Flyer III,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,2,7/12/1912,6:30,"AtlantiCity, New Jersey",Military - U.S. Navy,Dirigible,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,3,8/6/1913,NaN,"Victoria, British Columbia, Canada",Private,Curtiss seaplane,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,4,9/9/1913,18:30,Over the North Sea,Military - German Navy,Zeppelin L-1 (airship),20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,5,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,Zeppelin L-2 (airship),30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [8]:
#change column names to match db tables columns
clean_aircrashes_db = airplane_crashes_df.rename(columns={'Date':'AC_DATE'})

clean_aircrashes_db.head()

,Ac_id,AC_DATE,Time,Location,Operator,Type,Aboard,Fatalities,Ground,Summary
0,1,9/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,Wright Flyer III,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,2,7/12/1912,6:30,"AtlantiCity, New Jersey",Military - U.S. Navy,Dirigible,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,3,8/6/1913,NaN,"Victoria, British Columbia, Canada",Private,Curtiss seaplane,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,4,9/9/1913,18:30,Over the North Sea,Military - German Navy,Zeppelin L-1 (airship),20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,5,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,Zeppelin L-2 (airship),30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [9]:
#change date column to date format to insert into postgres
clean_aircrashes_db['AC_DATE'] = pd.to_datetime(clean_aircrashes_db['AC_DATE'])
clean_aircrashes_db.info()

#Extracting the year from the date
clean_aircrashes_db['year'] = pd.DatetimeIndex(clean_aircrashes_db['AC_DATE']).year
# clean_aircrashes_db.head()

#reordering columns to sync with the postgres table
clean_aircrashes_new_db = clean_aircrashes_db[['Ac_id','AC_DATE','year','Time','Location','Operator','Type','Aboard','Fatalities','Ground','Summary']]
clean_aircrashes_new_db.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 10 columns):
Ac_id         5268 non-null int64
AC_DATE       5268 non-null datetime64[ns]
Time          3049 non-null object
Location      5248 non-null object
Operator      5250 non-null object
Type          5241 non-null object
Aboard        5246 non-null float64
Fatalities    5256 non-null float64
Ground        5246 non-null float64
Summary       4878 non-null object
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 411.6+ KB


,Ac_id,AC_DATE,year,Time,Location,Operator,Type,Aboard,Fatalities,Ground,Summary
0,1,1908-09-17,1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,Wright Flyer III,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,2,1912-07-12,1912,6:30,"AtlantiCity, New Jersey",Military - U.S. Navy,Dirigible,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,3,1913-08-06,1913,NaN,"Victoria, British Columbia, Canada",Private,Curtiss seaplane,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,4,1913-09-09,1913,18:30,Over the North Sea,Military - German Navy,Zeppelin L-1 (airship),20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,5,1913-10-17,1913,10:30,"Near Johannisthal, Germany",Military - German Navy,Zeppelin L-2 (airship),30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [10]:
#change column names to match db tables columns
clean_aircrashes_new_db = clean_aircrashes_new_db.rename(columns={'Ac_id':'ac_id','AC_DATE':'ac_date','Time':'time','Location':'location','Operator':'operator','Type':'type','Aboard':'aboard','Fatalities':'fatalities','Ground':'ground','Summary':'summary'})
clean_aircrashes_new_db.head()

,ac_id,ac_date,year,time,location,operator,type,aboard,fatalities,ground,summary
0,1,1908-09-17,1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,Wright Flyer III,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,2,1912-07-12,1912,6:30,"AtlantiCity, New Jersey",Military - U.S. Navy,Dirigible,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,3,1913-08-06,1913,NaN,"Victoria, British Columbia, Canada",Private,Curtiss seaplane,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,4,1913-09-09,1913,18:30,Over the North Sea,Military - German Navy,Zeppelin L-1 (airship),20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,5,1913-10-17,1913,10:30,"Near Johannisthal, Germany",Military - German Navy,Zeppelin L-2 (airship),30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [67]:
crash=clean_aircrashes_new_db.loc[(clean_aircrashes_new_db["operator"]== "Aerolift")& (clean_aircrashes_new_db["year"]==2009)]
crash

,ac_id,ac_date,year,time,location,operator,type,aboard,fatalities,ground,summary
5250,5251,2009-02-20,2009,4:10,"Luxor, Egypt",Aerolift,Antonov 12V,5.0,5.0,0.0,"While attemping to take off from Luxor, the ca..."
5252,5253,2009-03-09,2009,5:40,"Lake Victoria, Uganda",Aerolift,Ilyushin Il-76T,11.0,11.0,0.0,The cargo plane carrying and water purificatio...


In [61]:
count=clean_aircrashes_new_db["operator"].value_counts()
count

Aeroflot                                             179
Military - U.S. Air Force                            176
Air France                                            70
Deutsche Lufthansa                                    65
China National Aviation Corporation                   44
Air Taxi                                              44
United Air Lines                                      44
Military - U.S. Army Air Forces                       43
Pan American World Airways                            41
US Aerial Mail Service                                36
Military - U.S. Navy                                  36
American Airlines                                     36
Military - Royal Air Force                            36
Indian Airlines                                       34
Philippine Air Lines                                  33
KLM Royal Dutch Airlines                              33
Private                                               31
Aeropostale                    

In [64]:
enginetype=clean_aircrashes_new_db["type"].value_counts()
enginetype

Douglas DC-3                                334
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                74
Douglas C-47                                 62
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
Douglas C-47B                                29
De Havilland DH-4                            28
Douglas DC-6B                                27
Breguet 14                                   23
Curtiss C-46A                                21
Douglas C-47-DL                              20
Douglas DC-6                                 20
Antonov AN-24                                19
McDonnell Douglas DC-9-32                    19
Antonov AN-12                                19
Curtiss C-46                                 18
Embraer 110P1 Bandeirante                    18
Junkers F-13                            

In [43]:
#Connect to local database
rds_connection_string = "postgres:putka@143@localhost:5432/airlines_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.connect()

In [44]:
engine.table_names()

['airplane_crashes', 'worst_geo_regions', 'airlines_safety', 'worst_accidents']

In [45]:
#Inserting the dataframe data into the airplane_crashes table.
clean_aircrashes_new_db.to_sql(name='airplane_crashes', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "airplane_crashes_pkey"
DETAIL:  Key (ac_id)=(1) already exists.

[SQL: INSERT INTO airplane_crashes (ac_id, ac_date, year, time, location, operator, type, aboard, fatalities, ground, summary) VALUES (%(ac_id)s, %(ac_date)s, %(year)s, %(time)s, %(location)s, %(operator)s, %(type)s, %(aboard)s, %(fatalities)s, %(ground)s, %(summary)s)]
[parameters: ({'ac_id': 1, 'ac_date': datetime.datetime(1908, 9, 17, 0, 0), 'year': 1908, 'time': '17:18', 'location': 'Fort Myer, Virginia', 'operator': 'Military - U.S. Army', 'type': 'Wright Flyer III', 'aboard': 2.0, 'fatalities': 1.0, 'ground': 0.0, 'summary': 'During a demonstration flight, a U.S. Army flyer flown by Orville Wright nose-dived into the ground from a height of approximately 75 feet, killing L ... (209 characters truncated) ...  of control of the aircraft.  Orville Wright suffered broken ribs, pelvis and a leg.  Selfridge suffered a crushed

In [46]:
#import airlane safety csv into a datframe
airline_safety_csvfile = "Airlines_Safety.csv"

airline_safety_df = pd.read_csv(airline_safety_csvfile, error_bad_lines=False,warn_bad_lines=False)
airline_safety_df.head()

,as_id,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14
0,1,Aer Lingus,320906734,2,0,0,0,0,0
1,2,Aeroflot,1197672318,76,14,128,6,1,88
2,3,Aerolineas Argentinas,385803648,6,0,0,1,0,0
3,4,Aeromexico,596871813,3,1,64,5,0,0
4,5,Air Canada,1865253802,2,0,0,2,0,0


In [47]:
#Create new dataframe with select columns
airplane_safety_df = airline_safety_df[['as_id','airline','incidents_85_99','fatal_accidents_85_99','fatalities_85_99','incidents_00_14','fatal_accidents_00_14','fatalities_00_14']]
airplane_safety_df.head()


,as_id,airline,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14
0,1,Aer Lingus,2,0,0,0,0,0
1,2,Aeroflot,76,14,128,6,1,88
2,3,Aerolineas Argentinas,6,0,0,1,0,0
3,4,Aeromexico,3,1,64,5,0,0
4,5,Air Canada,2,0,0,2,0,0


In [48]:
#change column names to match db tables columns
clean_airsafety_db = airplane_safety_df.rename(columns={'airline':'airlines'})

clean_airsafety_db.head()

,as_id,airlines,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14
0,1,Aer Lingus,2,0,0,0,0,0
1,2,Aeroflot,76,14,128,6,1,88
2,3,Aerolineas Argentinas,6,0,0,1,0,0
3,4,Aeromexico,3,1,64,5,0,0
4,5,Air Canada,2,0,0,2,0,0


In [49]:
#Inserting the dataframe data into the airlines_safety table.
clean_airsafety_db.to_sql(name='airlines_safety', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "airlines_safety_pkey"
DETAIL:  Key (as_id)=(1) already exists.

[SQL: INSERT INTO airlines_safety (as_id, airlines, incidents_85_99, fatal_accidents_85_99, fatalities_85_99, incidents_00_14, fatal_accidents_00_14, fatalities_00_14) VALUES (%(as_id)s, %(airlines)s, %(incidents_85_99)s, %(fatal_accidents_85_99)s, %(fatalities_85_99)s, %(incidents_00_14)s, %(fatal_accidents_00_14)s, %(fatalities_00_14)s)]
[parameters: ({'as_id': 1, 'airlines': 'Aer Lingus', 'incidents_85_99': 2, 'fatal_accidents_85_99': 0, 'fatalities_85_99': 0, 'incidents_00_14': 0, 'fatal_accidents_00_14': 0, 'fatalities_00_14': 0}, {'as_id': 2, 'airlines': 'Aeroflot', 'incidents_85_99': 76, 'fatal_accidents_85_99': 14, 'fatalities_85_99': 128, 'incidents_00_14': 6, 'fatal_accidents_00_14': 1, 'fatalities_00_14': 88}, {'as_id': 3, 'airlines': 'Aerolineas Argentinas', 'incidents_85_99': 6, 'fatal_accidents_85_99': 0, 'fatali

In [50]:
#import worst geo locations csv into a datframe
worst_geo_csvfile = "worst_geo_regions.csv"

worst_geo_df = pd.read_csv(worst_geo_csvfile, error_bad_lines=False,warn_bad_lines=False)
worst_geo_df.head()

,geo_id,country,accidents,fatalities,ground fat.
0,1,USA,847,10794,133
1,2,Russia,525,8453,25
2,3,Canada,189,1796,2
3,4,Brazil,188,2729,68
4,5,Colombia,183,2932,28


In [51]:
#change column names to match db tables columns
clean_geo_db = worst_geo_df.rename(columns={'ground fat.':'ground_fatalities'})

clean_geo_db.head()

,geo_id,country,accidents,fatalities,ground_fatalities
0,1,USA,847,10794,133
1,2,Russia,525,8453,25
2,3,Canada,189,1796,2
3,4,Brazil,188,2729,68
4,5,Colombia,183,2932,28


In [52]:
#Inserting the dataframe data into the airplane_crashes table.
clean_geo_db.to_sql(name='worst_geo_regions', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "worst_geo_regions_pkey"
DETAIL:  Key (geo_id)=(1) already exists.

[SQL: INSERT INTO worst_geo_regions (geo_id, country, accidents, fatalities, ground_fatalities) VALUES (%(geo_id)s, %(country)s, %(accidents)s, %(fatalities)s, %(ground_fatalities)s)]
[parameters: ({'geo_id': 1, 'country': 'USA', 'accidents': 847, 'fatalities': 10794, 'ground_fatalities': 133}, {'geo_id': 2, 'country': 'Russia', 'accidents': 525, 'fatalities': 8453, 'ground_fatalities': 25}, {'geo_id': 3, 'country': 'Canada', 'accidents': 189, 'fatalities': 1796, 'ground_fatalities': 2}, {'geo_id': 4, 'country': 'Brazil', 'accidents': 188, 'fatalities': 2729, 'ground_fatalities': 68}, {'geo_id': 5, 'country': 'Colombia', 'accidents': 183, 'fatalities': 2932, 'ground_fatalities': 28}, {'geo_id': 6, 'country': 'U.K.', 'accidents': 109, 'fatalities': 1296, 'ground_fatalities': 11}, {'geo_id': 7, 'country': 'France', 'accidents

In [53]:
#import worst accidents csv into a datframe
worst_acc_csvfile = "worst_accidents.csv"

worst_acc_df = pd.read_csv(worst_acc_csvfile, error_bad_lines=False,warn_bad_lines=False)
worst_acc_df.head()

,Acc_id,Fatal,Date,Location,Carrier,Type
0,1,2907,9/11/2001,"New York City, New York",American /United Airlines,B767 / B767
1,2,583,3/27/1977,"Tenerife, Canary Islands",Pan Am / KLM,B747 / B747
2,3,520,8/12/1985,"Mt. Osutaka, Japan",Japan Air Lines,B747
3,4,349,11/12/1996,"New Delhi, India",Saudi / Kazastan,B747 / Il76
4,5,346,3/3/1974,"Bois d' Ermenonville, France",Turkish Airlines,DC10


In [54]:
#change column names to match db tables columns
clean_worst_acc_db = worst_acc_df.rename(columns={'Acc_id':'acc_id','Fatal':'fatalities','Date':'acc_date','Location':'location','Carrier':'carrier','Type':'type'})

clean_worst_acc_db.head()

,acc_id,fatalities,acc_date,location,carrier,type
0,1,2907,9/11/2001,"New York City, New York",American /United Airlines,B767 / B767
1,2,583,3/27/1977,"Tenerife, Canary Islands",Pan Am / KLM,B747 / B747
2,3,520,8/12/1985,"Mt. Osutaka, Japan",Japan Air Lines,B747
3,4,349,11/12/1996,"New Delhi, India",Saudi / Kazastan,B747 / Il76
4,5,346,3/3/1974,"Bois d' Ermenonville, France",Turkish Airlines,DC10


In [55]:
#change date column to date format to insert into postgres
clean_worst_acc_db['acc_date'] = pd.to_datetime(clean_worst_acc_db['acc_date'])
clean_worst_acc_db.info()

# Create new dataframe with select columns
clean_worst_acc_db = clean_worst_acc_db[['acc_id','fatalities','acc_date','location','carrier','type']]
clean_worst_acc_db.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
acc_id        100 non-null int64
fatalities    100 non-null int64
acc_date      100 non-null datetime64[ns]
location      100 non-null object
carrier       100 non-null object
type          100 non-null object
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 4.8+ KB


,acc_id,fatalities,acc_date,location,carrier,type
0,1,2907,2001-09-11,"New York City, New York",American /United Airlines,B767 / B767
1,2,583,1977-03-27,"Tenerife, Canary Islands",Pan Am / KLM,B747 / B747
2,3,520,1985-08-12,"Mt. Osutaka, Japan",Japan Air Lines,B747
3,4,349,1996-11-12,"New Delhi, India",Saudi / Kazastan,B747 / Il76
4,5,346,1974-03-03,"Bois d' Ermenonville, France",Turkish Airlines,DC10


In [56]:
#Inserting the dataframe data into the worst_accidents table.
clean_worst_acc_db.to_sql(name='worst_accidents', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "worst_accidents_pkey"
DETAIL:  Key (acc_id)=(1) already exists.

[SQL: INSERT INTO worst_accidents (acc_id, fatalities, acc_date, location, carrier, type) VALUES (%(acc_id)s, %(fatalities)s, %(acc_date)s, %(location)s, %(carrier)s, %(type)s)]
[parameters: ({'acc_id': 1, 'fatalities': 2907, 'acc_date': datetime.datetime(2001, 9, 11, 0, 0), 'location': 'New York City, New York', 'carrier': 'American /United Airlines', 'type': 'B767 / B767'}, {'acc_id': 2, 'fatalities': 583, 'acc_date': datetime.datetime(1977, 3, 27, 0, 0), 'location': 'Tenerife, Canary Islands', 'carrier': 'Pan Am / KLM', 'type': 'B747 / B747'}, {'acc_id': 3, 'fatalities': 520, 'acc_date': datetime.datetime(1985, 8, 12, 0, 0), 'location': 'Mt. Osutaka, Japan', 'carrier': 'Japan Air Lines', 'type': 'B747'}, {'acc_id': 4, 'fatalities': 349, 'acc_date': datetime.datetime(1996, 11, 12, 0, 0), 'location': 'New Delhi, India', 'car

In [57]:
#Placing the data into a dataframe
#Selecting the columns from airplane_crashes table
airplane_crashes = pd.read_sql("select a.ac_id,a.fatalities,a.aboard\
           from airplane_crashes a;",conn)

airplane_crashes

,ac_id,fatalities,aboard
0,1,1.0,2.0
1,2,5.0,5.0
2,3,1.0,1.0
3,4,14.0,20.0
4,5,30.0,30.0
5,6,21.0,41.0
6,7,19.0,19.0
7,8,20.0,20.0
8,9,22.0,22.0
9,10,19.0,19.0
